In [6]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import re
import lxml
import cchardet
import time
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

D:\Downloads\Anaconda\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


The following document is from the open data of the Department of Conservation (DOC) in New Zealand. As seen below, it provides 3,262 unique trail names with global positioning

In [2]:
doc = pd.read_csv('data/DOC_Tracks.csv')
doc.head()
doc.shape
doc.info()

,OBJECTID,CAPTURE_ACCURACY,EQUIPMENT,DESCRIPTION,STATUS,OBJECT_TYPE_DESCRIPTION,GlobalID,SHAPE_Length
0,521495,2.0,100044818,TAUHERENIKAU GORGE TRACK,OPEN,Tramping Track,{3AAF81F3-73B3-4F88-A796-17B07DFF1A81},2310.732688
1,521496,2.0,100044819,MOUNT HOLDSWORTH TO JUMBO HUT,OPEN,Easy Tramping Track,{E84ABC5A-4EBE-4AE8-859E-48D9292A851E},3761.813276
2,521497,2.0,100044820,RAINGAUGE SPUR,OPEN,Easy Tramping Track,{83173518-A632-4A4D-8076-092A7851298F},2129.463229
3,521498,2.0,100044821,BARRA TRACK - THE PINES TO BLAKE STREAM,OPEN,Tramping Track,{96C9A611-59BA-4A5B-AD9B-ACAC7F3AC152},2046.447855
4,521499,2.0,100044822,EAST HOLDSWORTH TRACK,OPEN,Tramping Track,{E460CE11-526E-4DF0-99BF-9BF5EB00D38F},3508.240620


(3262, 8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3262 entries, 0 to 3261
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   OBJECTID                 3262 non-null   int64  
 1   CAPTURE_ACCURACY         3260 non-null   float64
 2   EQUIPMENT                3262 non-null   int64  
 3   DESCRIPTION              3262 non-null   object 
 4   STATUS                   3262 non-null   object 
 5   OBJECT_TYPE_DESCRIPTION  3262 non-null   object 
 6   GlobalID                 3262 non-null   object 
 7   SHAPE_Length             3262 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 204.0+ KB


Need some trail information for these tracks!

In [27]:
df = pd.read_csv('data/WalkingKiwi_Tracks2.csv', index_col=0)


,name,type,time_h,length_km,lng,lat,url,coordinates,trackElevation,maxElevation,minElevation,totalAscent
0,L. Wairoa Road End to Barrys Flat,Tramping Track,11.650000,11.51,173.096578,-41.494357,-l-wairoa-road-end-to-barrys-flat,"173.096578,-41.494357 173.096810,-41.494361 17...",192.003540039000000660962541587650775909423828...,407.1,178.5,228.6
1,Ngatuhoa Stream Track,Tramping Track,2.716667,6.45,176.006116,-37.927966,-ngatuhoa-stream-track,"176.006116,-37.927966 176.006240,-37.927909 17...",325.382751464839998334355186671018600463867187...,461.0,318.8,142.2
2,(coastal) Smoothwater-staffordhut Route,Route,21.133333,5.84,168.594405,-43.975153,-coastal-smoothwaterstaffordhut-route,"168.594405,-43.975153 168.594300,-43.975123 16...","1,0.317498326301569988583395343084703199565410...",37.6,0.0,37.6
3,(inland) Smoothwater-staffordhut Route,Route,2.983333,6.99,168.545144,-43.996206,-inland-smoothwaterstaffordhut-route,"168.545144,-43.996206 168.545165,-43.996220 16...",2.98784279823299980805018094542901962995529174...,253.1,1.3,251.8
4,1000 Acres (1000 Ac Jcn to Pp Hut),Tramping Track,23.100000,4.42,172.331671,-41.649679,1000-acres-1000-ac-jcn-to-pp-hut,"172.331671,-41.649679 172.330260,-41.646983 17...","345.5771484375,476.229125976559998889570124447...",1099.0,345.6,753.4


['-l-wairoa-road-end-to-barrys-flat', '-ngatuhoa-stream-track', '-coastal-smoothwaterstaffordhut-route', '-inland-smoothwaterstaffordhut-route', '1000-acres-1000-ac-jcn-to-pp-hut']


In [577]:
df.maxElevation[len(maxElevation)-1]

821.5

In [578]:
len(maxElevation)
maxElevation[-1]

2877

'\n1036.5m'

In [618]:
[len(maxElevation):])

Scrape Completed. Lists Appended.


,name,type,time_h,length_km,lng,lat,url,coordinates,trackElevation,maxElevation,minElevation,totalAscent,maxElevation2,length2,type2,totalAscent2,time2
0,L. Wairoa Road End to Barrys Flat,Tramping Track,11.650000,11.51,173.096578,-41.494357,-l-wairoa-road-end-to-barrys-flat,"173.096578,-41.494357 173.096810,-41.494361 17...",192.003540039000000660962541587650775909423828...,407.1,178.5,228.6,\n407.1m,11.51 km,one way,785.8m,7 hr 4 min
1,Ngatuhoa Stream Track,Tramping Track,2.716667,6.45,176.006116,-37.927966,-ngatuhoa-stream-track,"176.006116,-37.927966 176.006240,-37.927909 17...",325.382751464839998334355186671018600463867187...,461.0,318.8,142.2,\n461m,6.45 km,one way,340.1m,3 hr 48 min
2,(coastal) Smoothwater-staffordhut Route,Route,21.133333,5.84,168.594405,-43.975153,-coastal-smoothwaterstaffordhut-route,"168.594405,-43.975153 168.594300,-43.975123 16...","1,0.317498326301569988583395343084703199565410...",37.6,0.0,37.6,\n37.6m,5.84 km,one way,214m,2 hr 18 min
3,(inland) Smoothwater-staffordhut Route,Route,2.983333,6.99,168.545144,-43.996206,-inland-smoothwaterstaffordhut-route,"168.545144,-43.996206 168.545165,-43.996220 16...",2.98784279823299980805018094542901962995529174...,253.1,1.3,251.8,\n253.1m,6.99 km,one way,387.9m,2 hr 59 min
4,1000 Acres (1000 Ac Jcn to Pp Hut),Tramping Track,23.100000,4.42,172.331671,-41.649679,1000-acres-1000-ac-jcn-to-pp-hut,"172.331671,-41.649679 172.330260,-41.646983 17...","345.5771484375,476.229125976559998889570124447...",1099.0,345.6,753.4,\n1099m,4.42 km,one way,679.6m,3 hr 21 min


(3085, 17)